<a href="https://colab.research.google.com/github/smithkanzariya/Spam_Classifier/blob/master/Spam_Classifier_Alternative_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [75]:
#Loading the dataset
import pandas as pd 

messages = pd.read_csv("https://raw.githubusercontent.com/smithkanzariya/Spam_Classifier/master/SMSSpamCollection", sep='\t', names = ["label", "message"])

In [76]:
messages.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [77]:
#Data cleaning and preprocessing
import re 
import nltk 
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer

wordnet = WordNetLemmatizer()
corpus = []
for i in range(len(messages)):
    reviews = re.sub("[^a-zA-Z0-9]", ' ', messages['message'][i])
    reviews = reviews.lower()
    reviews = reviews.split()
    reviews = [wordnet.lemmatize(word) for word in reviews if word not in set(stopwords.words('english'))]
    reviews = " ".join(reviews)
    corpus.append(reviews)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [78]:
corpus[:5]

['go jurong point crazy available bugis n great world la e buffet cine got amore wat',
 'ok lar joking wif u oni',
 'free entry 2 wkly comp win fa cup final tkts 21st may 2005 text fa 87121 receive entry question std txt rate c apply 08452810075over18',
 'u dun say early hor u c already say',
 'nah think go usf life around though']

In [79]:
#Creating the Bag of Words Model
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X = cv.fit_transform(corpus).toarray() 

features = cv.get_feature_names()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [80]:
len(features)

8050

In [81]:
#Creating a TF-IDF Model
#from sklearn.feature_extraction.text import TfidfVectorizer
#cv = TfidfVectorizer()
#X = cv.fit_transform(corpus).toarray()    

In [82]:
X.shape

(5572, 8050)

In [83]:
Y = pd.get_dummies(messages['label'], drop_first=True)

In [84]:
Y

,spam
0,0
1,0
2,1
3,0
4,0
...,...
5567,1
5568,0
5569,0
5570,0


In [85]:
#Creating Train and Test Set
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.20, random_state = 0)

In [86]:
#Training Model using Naive Bayes classifier
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(X_train, Y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MultinomialNB()

In [87]:
#Prediting Output
Y_pred = model.predict(X_test)

In [88]:
#Confusion metrix
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(Y_test, Y_pred)
acc = accuracy_score(Y_test, Y_pred)

In [89]:
cm

array([[942,  13],
       [  5, 155]])

In [90]:
acc

0.9838565022421525

In [91]:
from sklearn.metrics import classification_report

print(classification_report(Y_test, Y_pred))


              precision    recall  f1-score   support

           0       0.99      0.99      0.99       955
           1       0.92      0.97      0.95       160

    accuracy                           0.98      1115
   macro avg       0.96      0.98      0.97      1115
weighted avg       0.98      0.98      0.98      1115



In [92]:
emails = ['Hey mohan, can we get together to watch footbal game tomorrow?',
    'Upto 20% discount on parking, exclusive offer just for you. Dont miss this reward!']

email_count = cv.transform(emails)
model.predict(email_count)

array([0, 1], dtype=uint8)